In [1]:
import pandas as pd
df = pd.read_csv('./ArticlesApril2018.csv')

In [2]:
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print(df.columns)
print('열의 개수: ', len(df.columns ))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
열의 개수:  15


In [5]:
df['headline'].isnull().values.any()

False

In [6]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [9]:
headline = [n for n in headline if n != "Unknown"]
len(headline)
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [8]:
from string import punctuation
def repreprocessing(s):
    s = s.encode('utf8').decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
# 단어 집합 만들기
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [12]:
# 훈련 데이터 만들기
# 정수 인코딩을 수행하는 동시에 하나의 문장을 여러 줄로 분해 (하나의 문장을 예측하기 위해 이전의 단어를 모두 참고하고자 하는 의도)

sequences = list()

for line in text: # 1214개의 데이터에 대해서 데이터를 1개씩 가져온다
    encoded = t.texts_to_sequences([line])[0] # 각 데이터에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [13]:
# 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만들기
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key
    
index_to_word[582]
# 582라는 인덱스는 offer라는 단어였음

'offer'

In [14]:
max_len = max(len(l) for l in sequences)
print(max_len)

24


In [15]:
from keras.preprocessing.sequence import pad_sequences
sequences  = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

Using TensorFlow backend.


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [17]:
print(y[:3])

[ 269  371 1115]


In [18]:
from keras.utils import to_categorical
# y 데이터에 대한 원핫 인코딩
y = to_categorical(y, num_classes = vocab_size)

In [19]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1)) # y데이터를 제거했으므로 -1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
model.fit(X, y, epochs=200, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 10s - loss: 7.6283 - acc: 0.0279
Epoch 2/200
 - 10s - loss: 7.1117 - acc: 0.0295
Epoch 3/200
 - 10s - loss: 6.9687 - acc: 0.0356
Epoch 4/200
 - 11s - loss: 6.8246 - acc: 0.0406
Epoch 5/200
 - 12s - loss: 6.6525 - acc: 0.0447
Epoch 6/200
 - 11s - loss: 6.4613 - acc: 0.0464
Epoch 7/200
 - 11s - loss: 6.2600 - acc: 0.0545
Epoch 8/200
 - 11s - loss: 6.0548 - acc: 0.0577
Epoch 9/200
 - 11s - loss: 5.8539 - acc: 0.0615
Epoch 10/200
 - 11s - loss: 5.6648 - acc: 0.0722
Epoch 11/200
 - 11s - loss: 5.4840 - acc: 0.0752
Epoch 12/200
 - 11s - loss: 5.3108 - acc: 0.0813
Epoch 13/200
 - 12s - loss: 5.1469 - acc: 0.0911
Epoch 14/200
 - 12s - loss: 4.9882 - acc: 0.0979
Epoch 15/200
 - 12s - loss: 4.8372 - acc: 0.1069
Epoch 16/200
 - 12s - loss: 4.6942 - acc: 0.1215
Epoch 17/200
 - 12s - loss: 4.5528 - acc: 0.1393
Epoch 18/200
 - 12s - loss: 4.4168 - acc: 0.1555
Epoch 1

Epoch 156/200
 - 11s - loss: 0.2934 - acc: 0.9161
Epoch 157/200
 - 11s - loss: 0.2930 - acc: 0.9167
Epoch 158/200
 - 11s - loss: 0.2908 - acc: 0.9170
Epoch 159/200
 - 11s - loss: 0.2901 - acc: 0.9179
Epoch 160/200
 - 12s - loss: 0.2891 - acc: 0.9170
Epoch 161/200
 - 12s - loss: 0.2886 - acc: 0.9173
Epoch 162/200
 - 12s - loss: 0.2897 - acc: 0.9158
Epoch 163/200
 - 13s - loss: 0.2901 - acc: 0.9161
Epoch 164/200
 - 13s - loss: 0.2873 - acc: 0.9157
Epoch 165/200
 - 13s - loss: 0.2906 - acc: 0.9157
Epoch 166/200
 - 13s - loss: 0.3050 - acc: 0.9148
Epoch 167/200
 - 13s - loss: 0.3105 - acc: 0.9130
Epoch 168/200
 - 13s - loss: 0.2908 - acc: 0.9172
Epoch 169/200
 - 13s - loss: 0.2831 - acc: 0.9181
Epoch 170/200
 - 13s - loss: 0.2813 - acc: 0.9168
Epoch 171/200
 - 14s - loss: 0.2816 - acc: 0.9158
Epoch 172/200
 - 15s - loss: 0.2800 - acc: 0.9175
Epoch 173/200
 - 14s - loss: 0.2802 - acc: 0.9161
Epoch 174/200
 - 14s - loss: 0.2796 - acc: 0.9153
Epoch 175/200
 - 14s - loss: 0.2815 - acc: 0.9161


In [20]:
def sentence_generation(model, t, current_word, n):
    # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word= current_word
    # 처음 들어온 단어도 마지막에 출력하기 위해 저장
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 단어)에 대해서 Y를 입력하고
        # Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word
    # 전체 반복
    sentence = init_word + sentence
    return sentence

In [21]:
print(sentence_generation(model, t, 'i', 10))

i want to be rich and im not sorry case of
